# Projeto 4 - Reconhecimento Facial com Inteligência Artificial

Aplicação de modelo de IA para reconhecimento de imagens relacionadas ao estado emocional de pessoas, como raiva e felicidade por exemplo. O desafio do modelo, além do alto volume de dados, é reconhecer a partir da face humana o sentimento que a pessoa está no momento. Para isso, será usado um banco de imagens público disponibilizado pelo Kaggle.

Fonte dos dados: 

## 1. Instalando e Carregando os Pacotes

In [1]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

Versão da Linguagem Python Usada Neste Jupyter Notebook: 3.9.13


In [2]:
# Instala pacote timm
!pip install -q timm

In [3]:
# Instala Pytorch
!pip install -q torch==1.13.0

In [4]:
# Instala Torchvision
!pip install -q torchvision==0.14.0

In [5]:
# Carga de pacotes

# Pacote timm
import timm 

# Carga de imagens
from PIL import Image

# Visualização e manipulação de dados
import numpy as np 
import matplotlib.pyplot as plt

# Pytorch
import torch 
from torch import nn 
from torch.utils.data import DataLoader

# Torchvision
from torchvision.datasets import ImageFolder
from torchvision import transforms as T 
from torchvision.transforms.transforms import ToTensor

# Barra de progressão
from tqdm import tqdm 

# Pacotes para o relatório de hardware
import gc
import types
import pkg_resources

In [6]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Projeto 4 - Reconhecimento Facial" --iversions

Author: Projeto 4 - Reconhecimento Facial

torch      : 1.13.0
timm       : 0.9.2
matplotlib : 3.5.2
numpy      : 1.21.5
PIL        : 9.2.0
torchvision: 0.14.0



In [7]:
# Relatório completo

# Verificando o dispositivo
processing_device = "cuda" if torch.cuda.is_available() else "cpu"

# Verificando se GPU pode ser usada (isso depende da plataforma CUDA estar instalada)
torch_aval = torch.cuda.is_available()

# Labels para o relatório de verificação
lable_1 = 'Visão Geral do Ambiente'
lable_2 = 'Se NVIDIA-SMI não for encontrado, então CUDA não está disponível'
lable_3 = 'Fim da Checagem'

# Função para verificar o que está importado nesta sessão
def get_imports():

    for name, val in globals().items():
        if isinstance(val, types.ModuleType):
            name = val.__name__.split(".")[0]

        elif isinstance(val, type):            
            name = val.__module__.split(".")[0]

        poorly_named_packages = {"PIL": "Pillow", "sklearn": "scikit-learn"}

        if name in poorly_named_packages.keys():
            name = poorly_named_packages[name]

        yield name

# Imports nesta sessão
imports = list(set(get_imports()))

# Loop para verificar os requerimentos
requirements = []
for m in pkg_resources.working_set:
    if m.project_name in imports and m.project_name!="pip":
        requirements.append((m.project_name, m.version))
        
# Pasta com os dados (quando necessário)
pasta_dados = r'images'

print(f'{lable_1:-^100}')
print()
print(f"Device:", processing_device)
print(f"Pasta de Dados: ", pasta_dados)
print(f"Versões dos Pacotes Requeridos: ", requirements)
print(f"Dispositivo Que Será Usado Para Treinar o Modelo: ", processing_device)
print(f"CUDA Está Disponível? ", torch_aval)
print("Versão do PyTorch: ", torch.__version__)
print()
print(f'{lable_2:-^100}\n')
!nvidia-smi
gc.collect()
print()
print(f"Limpando a Memória da GPU (se disponível): ", torch.cuda.empty_cache())
print("\nModelo da GPU:")
# Modelo da GPU usada
!nvidia-smi --query-gpu=name --format=csv,noheader
print(f'\n{lable_3:-^100}')

--------------------------------------Visão Geral do Ambiente---------------------------------------

Device: cpu
Pasta de Dados:  images
Versões dos Pacotes Requeridos:  [('Pillow', '9.2.0'), ('tqdm', '4.64.1'), ('matplotlib', '3.5.2'), ('numpy', '1.21.5'), ('torch', '1.13.0'), ('torchvision', '0.14.0'), ('timm', '0.9.2')]
Dispositivo Que Será Usado Para Treinar o Modelo:  cpu
CUDA Está Disponível?  False
Versão do PyTorch:  1.13.0+cpu

------------------Se NVIDIA-SMI não for encontrado, então CUDA não está disponível------------------


Limpando a Memória da GPU (se disponível):  None

Modelo da GPU:


'nvidia-smi' nÆo ‚ reconhecido como um comando interno
ou externo, um programa oper vel ou um arquivo em lotes.



------------------------------------------Fim da Checagem-------------------------------------------


'nvidia-smi' nÆo ‚ reconhecido como um comando interno
ou externo, um programa oper vel ou um arquivo em lotes.


In [8]:
# Device
device = processing_device

# Visualiza o device
print(device)

cpu


## 2. Organizando as Imagens em Disco

### 2.1 Define as pastas das imagens

In [9]:
# Pastas
train_img_folder_path = 'dados/train/'
valid_img_folder_path = 'dados/validation/'

### 2.2 Dataset Augmentation

Datasetaugmentation, também conhecido como "data augmentation", é uma técnica utilizada para aumentar a quantidade e a diversidade de dados disponíveis para treinar modelos de aprendizado de máquina, especialmente modelos de Deep Learning. 

O objetivo principal do data augmentation é melhorar a capacidade do modelo de generalizar e reduzir o overfitting, ou seja,  a  tendência  de  um  modelo  de  aprender  demais  com  os  dados  de  treinamento  e  não generalizar bem para novos dados não vistos.Data  augmentation  é  frequentemente  usado  quando  se  tem  um  conjunto  de  dados limitado ou desequilibrado, o que pode levar a problemas de desempenho e generalização. 

A ideia é criar novas amostras a partir das originais, aplicando transformações que preservem as características relevantes dos dados.

In [10]:
# Dataset Augmentation em treino
augmentation_treino = T.Compose([T.RandomHorizontalFlip(p = 0.5),
                                T.RandomRotation(degrees = (-20, +20)),
                                T.ToTensor()])

In [11]:
# Dataset Augmentation em validação
augmentation_valid = T.Compose([ToTensor()])

### 2.3 Prepara o dataset

In [12]:
# Prepara os dados de treino
dados_treino = ImageFolder(train_img_folder_path, transform = augmentation_treino)

In [13]:
# Prepara os dados de validação
dados_valid = ImageFolder(valid_img_folder_path, transform = augmentation_valid)

In [14]:
# Visualiza o total de imagens
print(f"Total de imagens no dataset de treino: {len(dados_treino)}")
print(f"Total de imagens no dataset de validação: {len(dados_valid)}")

Total de imagens no dataset de treino: 28821
Total de imagens no dataset de validação: 7066


In [15]:
# Índices das classes
print(dados_treino.class_to_idx)

{'angry': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'neutral': 4, 'sad': 5, 'surprise': 6}


## 3. Prepara os DataLoaders

In [16]:
# Hiperparâmetro
BATCH_SIZE = 32

In [17]:
# Criando os dataloaders
dl_treino = DataLoader(dados_treino, batch_size = BATCH_SIZE, shuffle = True)
dl_valid = DataLoader(dados_valid, batch_size = BATCH_SIZE, shuffle = True)

In [18]:
# Número de batches
print(f"Número de Batches (Lotes) no DataLoader de Treino: {len(dl_treino)}")
print(f"Número de Batches (Lotes) no DataLoader de Validação: {len(dl_valid)}")

Número de Batches (Lotes) no DataLoader de Treino: 901
Número de Batches (Lotes) no DataLoader de Validação: 221


In [19]:
# Extraindo 1 batch do DataLoader de treino
for images, labels in dl_treino:
    break; 

In [20]:
# Visualiza 1 batch
print(f"Shape de 1 Batch de Imagens: {images.shape}")
print(f"Shape de 1 Batch de Labels das Imagens do Batch: {labels.shape}")

Shape de 1 Batch de Imagens: torch.Size([32, 3, 48, 48])
Shape de 1 Batch de Labels das Imagens do Batch: torch.Size([32])


## 4. Modelagem

### 4.1 Cria o modelo como base

In [21]:
# Classe com arquitetura do modelo
class ModeloCNN(nn.Module):
    
    # Método construtor
    def __init__(self):
        
        # Inicializa a classe mãe (nn.Module)
        super(ModeloCNN, self).__init__()

        # Carrega o modelo pré-treinado
        self.eff_net = timm.create_model('efficientnet_b0', pretrained = True, num_classes = 7)

    # Método forward
    # Observe que labels = None, pois estamos fazendo exatamente as previsões dos labels
    def forward(self, images, labels = None):
        
        # Extrai as previsões (logits)
        logits = self.eff_net(images)

        # Calcula o erro do modelo
        if labels != None:
            loss = nn.CrossEntropyLoss()(logits, labels) 
            
            return logits, loss
        
        return logits 

In [22]:
# Cria uma instância da classe, um objeto
modelo = ModeloCNN()

In [23]:
# Envia o modelo para o device
modelo.to(device)

ModeloCNN(
  (eff_net): EfficientNet(
    (conv_stem): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): BatchNormAct2d(
      32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
      (drop): Identity()
      (act): SiLU(inplace=True)
    )
    (blocks): Sequential(
      (0): Sequential(
        (0): DepthwiseSeparableConv(
          (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (bn1): BatchNormAct2d(
            32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
            (drop): Identity()
            (act): SiLU(inplace=True)
          )
          (se): SqueezeExcite(
            (conv_reduce): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (act1): SiLU(inplace=True)
            (conv_expand): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (gate): Sigmoid()
          )
          (conv_pw): Conv2d(32, 16, kernel_size=(1, 

### 4.2 Funções para Loop de Treinamento e Avaliação

In [24]:
# Função para calcular a acurácia da classificação multiclasse
def multiclass_accuracy(y_pred, y_true):
    
    # Top classes
    top_p, top_class = y_pred.topk(1, dim = 1)
    
    # Classes iguais
    equals = top_class == y_true.view(*top_class.shape)
    
    # Retorna a média
    return torch.mean(equals.type(torch.FloatTensor))

In [25]:
# Função para o loop de treino
def treina_modelo(model, dataloader, optimizer, current_epo):
    
    # Coloca o modelo em modo de treino
    model.train()
    
    # Variáveis de controle
    total_loss = 0.0
    total_acc = 0.0
    
    # Mostra a barra de progressão durante o treino do modelo
    tk = tqdm(dataloader, desc = "epoch" + "[treino]" + str(current_epo + 1) + "/" + str(epochs))

    # Loop
    for t, data in enumerate(tk):
        
        # Extrai lote de imagens e labels
        images, labels = data
        
        # Envia imagens e labels para a memória do device
        images, labels = images.to(device), labels.to(device) 
        
        # Zera os gradientes
        optimizer.zero_grad()
        
        # Faz a previsão e retorna os logits
        logits, loss = model(images, labels)
        
        # Backpropagation
        loss.backward()
        optimizer.step()
        
        # Calcula o erro
        total_loss += loss.item()
        
        # Calcula a acurácia
        total_acc += multiclass_accuracy(logits,labels)
        
        # Atualiza a barra de progressão
        tk.set_postfix({'loss':'%6f' %float(total_loss/(t+1)), 'acc':'%6f'%float(total_acc/(t+1))})

    return total_loss/len(dataloader), total_acc/len(dataloader)

In [26]:
# Função para o loop de validação
def avalia_modelo(model, dataloader, current_epo):
    
    # Coloca o modelo em modo de avaliação
    model.eval()
    
    # Inicializa as variáveis de controle
    total_loss = 0.0
    total_acc = 0.0
    
    # Barra de progressão
    tk = tqdm(dataloader, desc = "epoch" + "[valid]" + str(current_epo+1) + "/" + str(epochs))

    # Loop
    for t, data in enumerate(tk):
        
        # Extrai um lote de imagens e labels
        images, labels = data
        
        # Envia imagens e labels para a memória do device
        images, labels = images.to(device),labels.to(device) 
        
        # Faz a previsão e retorna os logits
        logits, loss = model(images, labels)
        
        # Calcula erro e acurácia
        total_loss += loss.item()
        total_acc += multiclass_accuracy(logits,labels)
        
        # Atualiza a barra de progressão
        tk.set_postfix({'loss':'%6f' %float(total_loss/(t+1)), 'acc':'%6f'%float(total_acc/(t+1))})
  
    return total_loss/len(dataloader), total_acc/len(dataloader)

## 5. Treinamento e Avaliação

In [27]:
# Hiperparâmetros
LR = 0.001
epochs = 8

In [28]:
# Otimizador
optimizer = torch.optim.Adam(modelo.parameters(), lr = LR)

In [29]:
# Inicializa a variável de controle
best_valid_loss = np.Inf

In [30]:
%%time
for i in range(epochs):
    
    # Loop de treino
    train_loss, train_acc = treina_modelo(modelo, dl_treino, optimizer, i)
    
    # Loop de validação
    valid_loss, valid_acc = avalia_modelo(modelo, dl_valid, i)

    # Salva o melhor modelo
    if valid_loss < best_valid_loss:
        
        # Salva o modelo
        torch.save(modelo, 'modelo/melhor_modelo.pt')
        
        print("Salvando o melhor modelo em disco...")
        
        best_valid_loss = valid_loss

epoch[valid]1/8: 100%|██████████████████████████████████████████████████████████████████████| 221/221 [01:05<00:00,  3.37it/s, loss=1.396382, acc=0.472709]


Salvando o melhor modelo em disco...


epoch[valid]2/8: 100%|██████████████████████████████████████████████████████████████████████| 221/221 [00:26<00:00,  8.27it/s, loss=1.281387, acc=0.500816]


Salvando o melhor modelo em disco...


epoch[valid]3/8: 100%|██████████████████████████████████████████████████████████████████████| 221/221 [00:26<00:00,  8.20it/s, loss=1.156775, acc=0.560194]


...

Salvando o melhor modelo em disco...


epoch[valid]6/8: 100%|██████████████████████████████████████████████████████████████████████| 221/221 [00:26<00:00,  8.34it/s, loss=1.049651, acc=0.607597]


Salvando o melhor modelo em disco...


epoch[valid]8/8: 100%|██████████████████████████████████████████████████████████████████████| 221/221 [00:26<00:00,  8.25it/s, loss=1.024496, acc=0.614743]

Salvando o melhor modelo em disco...
Wall time: 57min 6s


In [31]:
## 6. Deploy do modelo

In [32]:
# Carrega o modelo treinado 
modelo_final = torch.load('modelo/melhor_modelo.pt')

In [33]:
modelo_final.eval()

ModeloCNN(
  (eff_net): EfficientNet(
    (conv_stem): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): BatchNormAct2d(
      32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
      (drop): Identity()
      (act): SiLU(inplace=True)
    )
    (blocks): Sequential(
      (0): Sequential(
        (0): DepthwiseSeparableConv(
          (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (bn1): BatchNormAct2d(
            32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
            (drop): Identity()
            (act): SiLU(inplace=True)
          )
          (se): SqueezeExcite(
            (conv_reduce): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (act1): SiLU(inplace=True)
            (conv_expand): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (gate): Sigmoid()
          )
          (conv_pw): Conv2d(32, 16, kernel_size=(1, 

In [34]:
# Carrega um batch de imagens
for image, label in dl_valid:
    break; 

In [35]:
# Previsão com o modelo treinado (a previsão é para um batch de dados)
with torch.no_grad():
    previsao = modelo_final(image)

In [36]:
# Captura uma imagem do lote de previsões
uma_imagem = previsao[6]

In [37]:
# São 7 previsões de classe para cada imagem. Aqui temos os logits
uma_imagem

tensor([ 1.1689, -2.0469,  1.5718, -0.0484,  1.0823,  2.2131, -2.3334])

In [38]:
# Converte os logits em probabilidades
probs = torch.nn.functional.softmax(uma_imagem, dim = -1)

# Visualiza o probs
print(probs)

tensor([0.1510, 0.0061, 0.2260, 0.0447, 0.1385, 0.4291, 0.0046])


In [39]:
# Extrai a maior probabilidade das previsões de classe da imagem
_, classe_predita = torch.max(probs, 0)

In [40]:
print(f"A classe predita para a imagem é: {classe_predita}")

A classe predita para a imagem é: 5


In [41]:
# Índices das classes
print(dados_valid.class_to_idx)

{'angry': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'neutral': 4, 'sad': 5, 'surprise': 6}
